<a href="https://colab.research.google.com/github/mrm8488/shared_colab_notebooks/blob/master/FUNIT_MonsterMirror__v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FUNIT Monster Mirror from selfie
## Take a selfie and map it to an animal face!
based on NVIDIA [Few-Shot Unsupervised Image-to-Image Translation](https://nvlabs.github.io/FUNIT/) and [MonsterMirror](https://github.com/leopd/MonsterMirror)

> Colab by [mrm8488](https://twitter.com/mrm8488) https://mrm8488.github.io

In [0]:
!git clone https://github.com/leopd/MonsterMirror.git

In [0]:
cd MonsterMirror/

In [0]:
!pip install -r requirements.txt

In [0]:
!pip install -U PyYAML

In [0]:
import cv2
import os
import numpy as np
import sys
from google.colab.patches import cv2_imshow

# Download face detector

In [0]:
cd pretrained-models

In [0]:
!wget https://github.com/leopd/SFD_pytorch/releases/download/0.3/s3fd_model.tgz

In [0]:
!tar xvzf s3fd_model.tgz
!rm s3fd_model.tgz

# Download GANimal model

In [0]:
!gdown https://drive.google.com/uc?id=1CsmSSWyMngtOLUL5lI-sEHVWc2gdJpF9

In [0]:
!tar xvzf pretrained.tar.gz
!rm pretrained.tar.gz
!rm animal119_gen_00100000.pt

In [0]:
cd /content/MonsterMirror

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# Take a selfie

In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [0]:
cd /content/MonsterMirror/monster_mirror/

In [0]:
__file__ = "localcam.py"

In [0]:
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath(__file__)), '..'))

In [0]:
import spooky

In [0]:
def flip_img(img:np.ndarray) -> np.ndarray:
    """Mirror image left-right
    """
    return img[:,::-1,:]

In [0]:
cd /content/MonsterMirror

# Choose an animal

In [0]:
#@title Choose an animal to map your face

option = 'meerkat' #@param ["meerkat", "pomeranian", "tiger"]
print(option)

In [0]:
spookifier = spooky.RoundRobinSpookifier(target_image_base="/content/MonsterMirror/photo.jpg", target_classes="/content/MonsterMirror/target-images/" + option +",/content/MonsterMirror/target-images/" +option+",/content/MonsterMirror/target-images/"+option,noise_drift=0.1,noise_speed=0.4, noise_mag=5)

In [0]:
im = cv2.imread("/content/MonsterMirror/photo.jpg")
img = spookifier.process_npimage(im, None)

In [0]:
flipped = flip_img(img)
cv2_imshow(flipped)